# Sliding Puzzle com Busca Gulosa

Como função heurística h(n) vamos usar Distância de Manhattan.

![1](imagens/1.png)

### Modelando o problema:

In [ ]:
import numpy as np

class SlidingPuzzle:
  def __init__(self, num_blocos):
    self.num_blocos = num_blocos

  def _encontra_posicao(self, estado, elemento):
    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        if elemento == estado[i, j]:
          return i, j
    return None, None

  def verifica_estados(self, no_atual, objetivo):
    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        if no_atual[i, j] != objetivo[i, j]:
          return False
    return True

  def expande_estados(self, no_atual):
    novos_estados = []

    linha, coluna = self._encontra_posicao(no_atual, 0)

    # Cima
    if linha != 0:
      nova_linha = linha - 1
      novo_estado = np.copy(no_atual)

      bloco_alvo = no_atual[nova_linha, coluna]
      novo_estado[nova_linha, coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Baixo
    if linha != self.num_blocos - 1:
      nova_linha = linha + 1
      novo_estado = np.copy(no_atual)

      bloco_alvo = no_atual[nova_linha, coluna]
      novo_estado[nova_linha, coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Esquerda
    if coluna != 0:
      nova_coluna = coluna - 1
      novo_estado = np.copy(no_atual)

      bloco_alvo = no_atual[linha, nova_coluna]
      novo_estado[linha, nova_coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    # Direita
    if coluna != self.num_blocos - 1:
      nova_coluna = coluna + 1
      novo_estado = np.copy(no_atual)

      bloco_alvo = no_atual[linha, nova_coluna]
      novo_estado[linha, nova_coluna] = 0
      novo_estado[linha, coluna] = bloco_alvo
      novos_estados.append(novo_estado)

    return novos_estados
            
  def distancia_Manhattan(self, no_atual, objetivo):

    '''
    Calcula a distância de Manhattan entre o estado no_atual e o estado objetivo
    Args: 
        - no_atual: estado no_atual do tabuleiro
        - objetivo: estado objetivo do problema
    Return:
        - Distância de Manhattan
    '''
    
    distancia_total = 0

    for i in range(self.num_blocos):
      for j in range(self.num_blocos):
        bloco_atual = no_atual[i, j]
        linha, coluna = self._encontra_posicao(objetivo, bloco_atual)
        
        distancia = abs(linha - i) + abs(coluna - j)
        distancia_total += distancia

    return distancia_total

### Modelando a Busca Gulosa:

In [ ]:
import heapq

class BuscaGulosa():
  def __init__(self, problema: SlidingPuzzle):
      self.problema = problema
        
  def _verifica_visitado(self, estado, estados_visitados):
    for i in estados_visitados:
      if self.problema.verifica_estados(i, estado):
        return True
    return False

    
  def busca(self, inicio, fim):
    '''
      Realiza a busca gulosa, armazenando os estados em uma FILA DE PRIORIDADES
      Args:
          - inicio: estado inicial do problema
          - fim: estado objetivo
      Return:
          - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
    '''
    
    '''
      OBS.: A distância de Manhattan é inversamente proporcional à prioridade, quanto menor a distância, maior
      a prioridade desse estado
    '''

    
    # fila: fila manipulada pela heapq.
    fila = []

    id = 0

    # (distancia, id, estado): tupla inserida na fila sendo:
    #   - posição 0 a heurística;
    #   - posição 1 a um identificador;
    #       Caso nao tenha esse identificador é gerado um erro se for inserido estados com heurísticas iguais. 
    #   - posição 2 o estado.
    heapq.heappush(fila, (0, id, inicio))

    solucao_encontrada = False
    estados_visitados = []
    cont_estados = 0

    while not len(fila) == 0:
      no_atual = heapq.heappop(fila)[2]
      estados_visitados.append(no_atual)

      if self.problema.verifica_estados(no_atual, fim):
        solucao_encontrada = True
        break

      else:
        cont_estados += 1
        novos_estados = self.problema.expande_estados(no_atual)
        for novo_estado in novos_estados:
          if not self._verifica_visitado(novo_estado, estados_visitados):
            id += 1
            distancia = self.problema.distancia_Manhattan(novo_estado, fim)
            heapq.heappush(fila, (distancia, id, novo_estado))

    return solucao_encontrada, estados_visitados, cont_estados

### Resolvendo o problema:

In [ ]:
problema = SlidingPuzzle(3)
bg = BuscaGulosa(problema)

inicio = np.array([[2, 8, 3],
                   [1, 6, 4],
                   [7, 0, 5]])

alvo = np.array([[1, 2, 3],
                 [8, 0, 4],
                 [7, 6, 5]])

solucao, visitados, passos = bg.busca(inicio, alvo)

from pprint import pprint

print(f'Solução encontrada? {solucao}')
print(f'Estados visitados:') 
pprint(visitados)
print(f'Quantidade de passos: {passos}')